# TensorTrade - Renderers and Plotly Visualization Chart
## Data Loading Function

In [1]:
# ipywidgets is required to run Plotly in Jupyter Notebook.
# Uncomment and run the following line to install it if required.

#!pip install ipywidgets

In [2]:
import ta

import pandas as pd

from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC
from tensortrade.oms.wallets import Wallet, Portfolio

%matplotlib inline

In [3]:
def load_csv(filename):
    df = pd.read_csv('data/' + filename, skiprows=1)
    df.drop(columns=['symbol', 'volume_btc'], inplace=True)

    # Fix timestamp form "2019-10-17 09-AM" to "2019-10-17 09-00-00 AM"
    df['date'] = df['date'].str[:14] + '00-00 ' + df['date'].str[-2:]

    # Convert the date column type from string to datetime for proper sorting.
    df['date'] = pd.to_datetime(df['date'])

    # Make sure historical prices are sorted chronologically, oldest first.
    df.sort_values(by='date', ascending=True, inplace=True)

    df.reset_index(drop=True, inplace=True)

    # Format timestamps as you want them to appear on the chart buy/sell marks.
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %I:%M %p')

    return df

In [4]:
df = load_csv('Coinbase_BTCUSD_1h.csv')
df.head()

,date,open,high,low,close,volume
0,2017-07-01 11:00 AM,2505.56,2513.38,2495.12,2509.17,287000.32
1,2017-07-01 12:00 PM,2509.17,2512.87,2484.99,2488.43,393142.50
2,2017-07-01 01:00 PM,2488.43,2488.43,2454.40,2454.43,693254.01
3,2017-07-01 02:00 PM,2454.43,2473.93,2450.83,2459.35,712864.80
4,2017-07-01 03:00 PM,2459.35,2475.00,2450.00,2467.83,682105.41


## Data Preparation
### Create the dataset features

In [5]:
dataset = ta.add_all_ta_features(df, 'open', 'high', 'low', 'close', 'volume', fillna=True)
dataset.head(3)

/Users/xichao.chen/miniconda3/envs/python38/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/xichao.chen/miniconda3/envs/python38/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,date,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2017-07-01 11:00 AM,2505.56,2513.38,2495.12,2509.17,287000.32,154659.537174,287000.32,0.538883,0.000000e+00,...,-23.055860,0.0,2509.170000,0.0,0.000000,0.000000,0.000000,-65.047642,0.000000,0.000000
1,2017-07-01 12:00 PM,2509.17,2512.87,2484.99,2488.43,393142.50,-141466.449196,-106142.18,-0.207995,-8.153775e+06,...,-87.883057,0.0,2500.178580,0.0,2.871568,0.574314,2.297254,-0.826568,-0.830003,-0.826568
2,2017-07-01 01:00 PM,2488.43,2488.43,2454.40,2454.43,693254.01,-833498.148276,-799396.19,-0.606888,-1.035618e+07,...,-99.949135,0.0,2480.168753,0.0,12.006246,2.860700,9.145546,-1.366323,-1.375743,-2.181598


### Create Chart Price History Data
Note: It is recommended to create the chart data *after* creating and cleaning the dataset to ensure one-to-one mapping between the historical prices data and the dataset.

In [6]:
price_history = dataset[['date', 'open', 'high', 'low', 'close', 'volume']]  # chart data
display(price_history.head(3))

dataset.drop(columns=['date', 'open', 'high', 'low', 'close', 'volume'], inplace=True)

,date,open,high,low,close,volume
0,2017-07-01 11:00 AM,2505.56,2513.38,2495.12,2509.17,287000.32
1,2017-07-01 12:00 PM,2509.17,2512.87,2484.99,2488.43,393142.50
2,2017-07-01 01:00 PM,2488.43,2488.43,2454.40,2454.43,693254.01


## Setup Trading Environment
### Create Data Feeds

In [15]:
bitfinex = Exchange("bitfinex", service=execute_order)(
    Stream.source(price_history['close'].tolist(), dtype="float").rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(bitfinex, 10000 * USD),
    Wallet(bitfinex, 10 * BTC),
])

with NameSpace("bitfinex"):
    streams = [Stream.source(dataset[c].tolist(), dtype="float").rename(c) for c in dataset.columns]

feed = DataFeed(streams)
feed.next()

{'bitfinex:/volume_adi': 154659.5371741516,
 'bitfinex:/volume_obv': 287000.32,
 'bitfinex:/volume_cmf': 0.5388828039430464,
 'bitfinex:/volume_fi': 0.0,
 'bitfinex:/volume_mfi': 50.0,
 'bitfinex:/volume_em': 0.0,
 'bitfinex:/volume_sma_em': 0.0,
 'bitfinex:/volume_vpt': -187039.68188942783,
 'bitfinex:/volume_nvi': 1000.0,
 'bitfinex:/volume_vwap': 2505.89,
 'bitfinex:/volatility_atr': 0.0,
 'bitfinex:/volatility_bbm': 2509.17,
 'bitfinex:/volatility_bbh': 2509.17,
 'bitfinex:/volatility_bbl': 2509.17,
 'bitfinex:/volatility_bbw': 0.0,
 'bitfinex:/volatility_bbp': 0.0,
 'bitfinex:/volatility_bbhi': 0.0,
 'bitfinex:/volatility_bbli': 0.0,
 'bitfinex:/volatility_kcc': 2505.89,
 'bitfinex:/volatility_kch': 2524.15,
 'bitfinex:/volatility_kcl': 2487.6299999999997,
 'bitfinex:/volatility_kcw': 1.457366444656407,
 'bitfinex:/volatility_kcp': 0.5898138006571786,
 'bitfinex:/volatility_kchi': 0.0,
 'bitfinex:/volatility_kcli': 0.0,
 'bitfinex:/volatility_dcl': 2495.12,
 'bitfinex:/volatility_

### Trading Environment Renderers
A renderer is a channel for the trading environment to output its current state. One or more renderers can be attached to the environment at the same time. For example, you can let the environment draw a chart and log to a file at the same time.

Notice that while all renderers can technically be used together, you need to select the best combination to avoid undesired results. For example, PlotlyTradingChart can work well with FileLogger but may not display well with ScreenLogger.

Renderer can be set by name (string) or class, single or list. Available renderers are:
* `'screenlog'` or `ScreenLogger`: Shows results on the screen.
* `'filelog'` or `FileLogger`: Logs results to a file.
* `'plotly'` or `PlotlyTradingChart`: A trading chart based on Plotly.

#### Examples:

* renderers = 'screenlog' (default)
* renderers = ['screenlog', 'filelog']
* renderers = ScreenLogger()
* renderers = ['screenlog', `FileLogger()`]
* renderers = [`FileLogger(filename='example.log')`]

Renderers can also be created and configured first then attached to the environment as seen in a following example.

### Trading Environment with a Single Renderer

In [14]:
import tensortrade.env.default as default

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer="screen-log",  # ScreenLogger used with default settings
    window_size=20
)

In [9]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)
agent.train(n_episodes=2, n_steps=200, render_interval=10)

====      AGENT ID: 474131c0-7c88-4e0e-80b2-16b3972da585      ====
[2021-03-08 10:30:40 PM] Episode: 1/2 Step: 131/200
[2021-03-08 10:30:41 PM] Episode: 1/2 Step: 141/200
[2021-03-08 10:30:41 PM] Episode: 1/2 Step: 151/200
[2021-03-08 10:30:42 PM] Episode: 1/2 Step: 161/200
[2021-03-08 10:30:43 PM] Episode: 1/2 Step: 171/200
[2021-03-08 10:30:44 PM] Episode: 1/2 Step: 181/200
[2021-03-08 10:30:45 PM] Episode: 1/2 Step: 191/200
[2021-03-08 10:30:46 PM] Episode: 1/2 Step: 201/200
[2021-03-08 10:30:47 PM] Episode: 2/2 Step: 11/200
[2021-03-08 10:30:47 PM] Episode: 2/2 Step: 21/200
[2021-03-08 10:30:48 PM] Episode: 2/2 Step: 31/200
[2021-03-08 10:30:49 PM] Episode: 2/2 Step: 41/200
[2021-03-08 10:30:50 PM] Episode: 2/2 Step: 51/200
[2021-03-08 10:30:51 PM] Episode: 2/2 Step: 61/200
[2021-03-08 10:30:52 PM] Episode: 2/2 Step: 71/200
[2021-03-08 10:30:52 PM] Episode: 2/2 Step: 81/200
[2021-03-08 10:30:53 PM] Episode: 2/2 Step: 91/200
[2021-03-08 10:30:54 PM] Episode: 2/2 Step: 101/200
[2021-

98905.8887729663

### Environment with Multiple Renderers
Create PlotlyTradingChart and FileLogger renderers. Configuring renderers is optional as they can be used with their default settings.

In [10]:
from tensortrade.env.default.renderers import PlotlyTradingChart, FileLogger

chart_renderer = PlotlyTradingChart(
    display=True,  # show the chart on screen (default)
    height=800,  # affects both displayed and saved file height. None for 100% height.
    save_format="html",  # save the chart to an HTML file
    auto_open_html=True,  # open the saved HTML chart in a new browser tab
)

file_logger = FileLogger(
    filename="example.log",  # omit or None for automatic file name
    path="training_logs"  # create a new directory if doesn't exist, None for no directory
)

### Environement with Multiple Renderers

With the plotly renderer you must provide an parameter called `renderer_feed`. This is a `DataFeed` instance that provides all the information that is required by a renderer to function.

In [11]:
renderer_feed = DataFeed([
    Stream.source(price_history[c].tolist(), dtype="float").rename(c) for c in price_history]
)

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    window_size=20,
    renderer_feed=renderer_feed,
    renderers=[
        chart_renderer, 
        file_logger
    ]
)

## Setup and Train DQN Agent
The green and red arrows shown on the chart represent buy and sell trades respectively. The head of each arrow falls at the trade execution price.

In [18]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)

# Set render_interval to None to render at episode ends only
agent.train(n_episodes=2, n_steps=200, render_interval=10)

====      AGENT ID: 27bb3139-2ab7-483b-a1ee-0e79df2da125      ====
[2021-03-08 10:33:45 PM] Episode: 1/2 Step: 131/200
[2021-03-08 10:33:46 PM] Episode: 1/2 Step: 141/200
[2021-03-08 10:33:47 PM] Episode: 1/2 Step: 151/200
[2021-03-08 10:33:48 PM] Episode: 1/2 Step: 161/200
[2021-03-08 10:33:49 PM] Episode: 1/2 Step: 171/200
[2021-03-08 10:33:50 PM] Episode: 1/2 Step: 181/200
[2021-03-08 10:33:50 PM] Episode: 1/2 Step: 191/200
[2021-03-08 10:33:51 PM] Episode: 1/2 Step: 201/200
[2021-03-08 10:33:52 PM] Episode: 2/2 Step: 11/200
[2021-03-08 10:33:53 PM] Episode: 2/2 Step: 21/200
[2021-03-08 10:33:54 PM] Episode: 2/2 Step: 31/200
[2021-03-08 10:33:55 PM] Episode: 2/2 Step: 41/200
[2021-03-08 10:33:56 PM] Episode: 2/2 Step: 51/200
[2021-03-08 10:33:57 PM] Episode: 2/2 Step: 61/200
[2021-03-08 10:33:58 PM] Episode: 2/2 Step: 71/200
[2021-03-08 10:33:58 PM] Episode: 2/2 Step: 81/200
[2021-03-08 10:33:59 PM] Episode: 2/2 Step: 91/200
[2021-03-08 10:34:00 PM] Episode: 2/2 Step: 101/200
[2021-

83983.09155005601

## Direct Performance and Net Worth Plotting
Alternatively, the final performance and net worth can be displayed using pandas via Matplotlib.

In [19]:
%matplotlib inline

portfolio.performance.plot()

AttributeError: 'NoneType' object has no attribute 'plot'

In [17]:
portfolio.performance.net_worth.plot()

AttributeError: 'NoneType' object has no attribute 'net_worth'